In [1]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import os
import time
from sklearn.decomposition import PCA
from scipy import signal
import math

#Reading Train and Dev Data
train_class_labels = []
train_data = []
with open("Synthetic_Dataset/train.txt") as f:
    temp = [[float(val) for val in line.strip().split(',')] for line in f]
    for i in temp:
        train_data.append(np.array([i[0],i[1]]))
        train_class_labels.append(int(i[2]-1))
train_data = np.array(train_data)

dev_class_labels = []
dev_data = []
with open("Synthetic_Dataset/dev.txt") as f:
    temp = [[float(val) for val in line.strip().split(',')] for line in f]
    for i in temp:
        dev_data.append(np.array([i[0],i[1]]))
        dev_class_labels.append(int(i[2]-1))
dev_data = np.array(dev_data)


In [23]:

clf = MLPClassifier(solver='adam', alpha=0,hidden_layer_sizes=(10,10), random_state=1, max_iter = 2000)
clf.fit(train_data, train_class_labels)

MLPClassifier(alpha=0, hidden_layer_sizes=(10, 10), max_iter=2000,
              random_state=1)

In [24]:
predictions = clf.predict(dev_data)

In [25]:
count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == dev_class_labels[i]:
        count_correct += 1
print(count_correct/len(dev_data))

1.0


In [64]:

classes = ['coast','forest','highway','mountain','opencountry']
train = []  
train_all = []
dev_all = []
dev = []
test = []
denoms = []
priors = np.zeros(len(classes))
train_labels = []
dev_labels = []
dev_img_label= []
train_img_label= []

train_imgs = []
dev_imgs = []
#Reading the Images of each class for train and Developement DataSets
for i,cls in enumerate(classes):
    lst = []
    dir_list = os.listdir('Features//'+cls+'//train')
    priors[i] = len(dir_list)
    for file in dir_list:
        lst.extend(np.loadtxt('Features/'+cls+'/train/' + file))
        concat = np.array([])
        for r in np.loadtxt('Features/'+cls+'/train/' + file):
            concat = np.concatenate((concat,r))
        train_imgs.append(concat)
        train_labels.extend([i]*36)
        train_img_label.extend([i])
    lst = np.array(lst)
    train.append(lst)
    train_all.extend(lst)

    lst = []
    lst2 = []
    dir_list = os.listdir('Features//'+cls+'//dev')
    for file in dir_list:
        lst.append(np.loadtxt('Features/'+cls+'/dev/' + file))
        lst2.extend(np.loadtxt('Features/'+cls+'/dev/' + file))
        concat = np.array([])
        for r in np.loadtxt('Features/'+cls+'/dev/' + file):
            concat = np.concatenate((concat,r))
        dev_imgs.append(concat)
        dev_labels.extend([i]*36)
        dev_img_label.extend([i])
    dev.append(np.array(lst))
    test.append(np.array(lst2))
    dev_all.extend(lst2)
# train_all = np.array(train_all)
# dev_all = np.array(dev_all)
#Mean Normalisation
mean_train = np.mean(train_all,axis=0)
maxs = np.max(train_all,axis=0)
mins = np.min(train_all,axis=0)
denoms = maxs - mins
train_all = (train_all-mean_train)/denoms
dev_all = (dev_all-mean_train)/denoms



#Principal Component Analysis, for reducing Dimensionality
pca = PCA(.99)
pca.fit(np.array(train_all))
train_all = pca.transform(train_all)
dev_all = pca.transform(dev_all)

train_all = np.array(train_all)
dev_all = np.array(dev_all)

#Principal Component Analysis, for reducing Dimensionality
pca = PCA(.9)
pca.fit(np.array(train_imgs))
train_imgs = pca.transform(train_imgs)
dev_imgs = pca.transform(dev_imgs)

train_imgs = np.array(train_imgs)
dev_imgs = np.array(dev_imgs)

In [65]:
print(train_imgs.shape)

(1210, 136)


In [3]:
clf = MLPClassifier(solver='sgd', activation="tanh",alpha=0,hidden_layer_sizes=(40,20,10), random_state=1, max_iter = 2000)
clf.fit(train_all, train_labels)

MLPClassifier(activation='tanh', alpha=0, hidden_layer_sizes=(40, 20, 10),
              max_iter=2000, random_state=1, solver='sgd')

In [12]:
predictions = clf.predict(dev_all)

In [16]:
predictions_img = []
for i in range(len(dev_img_label)):
    predictions_img.append(np.bincount(predictions[36*i:36*(i+1)]).argmax())
predictions_img = np.array(predictions_img)
print(predictions_img)

[0 3 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 4 0 4 3 0 0 2 0 3 4 0 0 0 0 0 0 0 2 0 1
 0 0 0 0 0 2 0 0 0 0 3 0 3 3 0 0 0 0 0 2 0 0 0 0 2 2 0 3 0 3 0 3 0 0 3 0 1
 4 1 4 3 4 1 1 1 1 1 1 1 1 1 4 1 1 1 1 1 1 1 1 1 1 3 3 1 3 3 1 1 1 1 3 1 1
 1 1 1 1 1 1 1 1 1 3 3 3 3 3 3 1 1 1 4 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2
 4 0 4 2 2 2 2 2 2 2 2 2 2 2 3 0 3 3 2 3 0 3 3 3 3 3 3 3 3 3 4 3 2 3 2 4 3
 0 3 0 0 0 4 3 3 3 3 0 3 1 3 3 1 3 3 3 2 0 3 3 2 3 1 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 4 3 3 1 1 3 1 3 1 3 1 1 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4
 3 3 3 1 3 3 3 0 3 1 1 4 4 1 4 1 1 1 1 0 4 0 3 3 4 1 4 4 3 3 3 1 4 1 1 1 3
 3 1 4 4 1 3 1 4 1 0 0 1 1 4 3 4 4 4 4 3 3 0 3 4 1 1 4 3 0 3 4 1 1 2 1 1 1
 1 1 4 1 1 4 4 1 3 1 0 3 1 4 0]


In [17]:
count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == dev_labels[i]:
        count_correct += 1
print(count_correct/len(dev_all))

0.4492337164750958


In [18]:
count_correct = 0
for i in range(len(predictions_img)):
    if predictions_img[i] == dev_img_label[i]:
        count_correct += 1
print(count_correct/len(dev_img_label))

0.5919540229885057


In [19]:
predictions = clf.predict(train_all)

count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == train_labels[i]:
        count_correct += 1
print(count_correct/len(train_all))

0.5517676767676768


In [20]:
predictions_img = []
for i in range(len(train_img_label)):
    predictions_img.append(np.bincount(predictions[36*i:36*(i+1)]).argmax())
predictions_img = np.array(predictions_img)
print(predictions_img)

count_correct = 0
for i in range(len(predictions_img)):
    if predictions_img[i] == train_img_label[i]:
        count_correct += 1
print(count_correct/len(train_img_label))

[0 0 0 ... 3 4 0]
0.7702479338842976


ANN with 828 vector

In [114]:
clf = MLPClassifier(solver='adam', activation="tanh",alpha=1,hidden_layer_sizes=(50), random_state=1, max_iter = 5000)
clf.fit(train_imgs, train_img_label)

MLPClassifier(activation='tanh', alpha=1, hidden_layer_sizes=50, max_iter=5000,
              random_state=1)

In [115]:
predictions = clf.predict(train_imgs)

In [116]:
count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == train_img_label[i]:
        count_correct += 1
print(count_correct/len(train_img_label))

0.9239669421487603


In [117]:
predictions = clf.predict(dev_imgs)
count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == dev_img_label[i]:
        count_correct += 1
print(count_correct/len(dev_img_label))

0.6293103448275862


ANN ISOLATED DIGITS

In [2]:
# Isolated Digits
digits = [1,2,5,9,'z']
train = []
dev = []

#Loading train and dev data
for digit in digits:
    lst = []
    dir_list = os.listdir('Isolated_Digits//'+str(digit)+'//train')
    for file in dir_list:
        if file.endswith('.mfcc'):
            lst.append(np.loadtxt('Isolated_Digits/'+str(digit)+'/train/' + file,skiprows = 1))

    train.append(np.array(lst,dtype=object))

    lst = []
    dir_list = os.listdir('Isolated_Digits//'+str(digit)+'//dev')
    for file in dir_list:
        if file.endswith('.mfcc'):
            lst.append(np.loadtxt('Isolated_Digits/'+str(digit)+'/dev/' + file,skiprows = 1))

    dev.append(np.array(lst,dtype=object))

count = 0
num_frames = 0
for cls in range(len(train)):
    for i in range(len(train[cls])):
        num_frames += len(train[cls][i])
        count += 1
avg_num_frames = math.floor(num_frames/count)

train_all = []
dev_all = []
train_labels = []
dev_labels = []
for cls in range(len(train)):
    for i in range(len(train[cls])):
        train_all.append(signal.resample(train[cls][i],avg_num_frames))
        train_labels.append(cls)

for cls in range(len(dev)):
    for i in range(len(dev[cls])):
        dev_all.append(signal.resample(dev[cls][i],avg_num_frames))
        dev_labels.append(cls)

train_all = np.array(train_all)
dev_all = np.array(dev_all)

mean_train = np.mean(train_all,axis=0)
maxs = np.max(train_all,axis=0)
mins = np.min(train_all,axis=0)
denoms = maxs - mins
train_all = (train_all-mean_train)/denoms
dev_all = (dev_all-mean_train)/denoms

In [3]:
train_extended = []
dev_extended = []
for i in range(len(train_all)):
    lst = []
    for j in range(len(train_all[i])):
        lst.extend(train_all[i][j])
    train_extended.append(np.array(lst))

for i in range(len(dev_all)):
    lst = []
    for j in range(len(dev_all[i])):
        lst.extend(dev_all[i][j])
    dev_extended.append(np.array(lst))

In [4]:
clf = MLPClassifier(solver='adam', activation="logistic",alpha=0.1,hidden_layer_sizes=(30,30), random_state=1, max_iter = 5000)
clf.fit(train_extended, train_labels)

MLPClassifier(activation='logistic', alpha=0.1, hidden_layer_sizes=(30, 30),
              max_iter=5000, random_state=1)

In [5]:
predictions = clf.predict(dev_extended)
count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == dev_labels[i]:
        count_correct += 1
print(count_correct/len(dev_labels))

0.9166666666666666


ANN telugu charr

In [7]:

# Handwriting Data
letters = ['a','bA','chA','lA','tA']
train = []
dev = []

#Loading train and dev data
for letter in letters:
    lst = []
    dir_list = os.listdir('Handwriting_Data//'+letter+'//train')
    for file in dir_list:
        temp = np.loadtxt('Handwriting_Data/'+letter+'/train/' + file)[1:]
        train_lst = np.array([temp[::2],temp[1::2]]).T
        train_lst = train_lst.tolist()
        #Here we add an extra feature as mentioned in the report.This feature is arctan(slope) which is basically the angle made by tangent with x axis
        #For reason we add this angle in radians and not the slope directly, please refer to the report
        for i in range(len(train_lst)-1): 
            if train_lst[i+1][0] != train_lst[i][0]: 
                train_lst[i].append(np.arctan((train_lst[i+1][1] - train_lst[i][1])/(train_lst[i+1][0] - train_lst[i][0])))
            else: #If x coordinates of consecutive points are same, then we cannot divide by x2-x1 and hence directly take arctan on +-np.inf
                if train_lst[i+1][1] - train_lst[i][1] > 0: #arctan(np.inf) if y2 - y1 > 0
                    train_lst[i].append(np.arctan(np.inf))
                else:
                    train_lst[i].append(np.arctan(-np.inf)) #arctan(-np.inf) if y1 - y2 > 0
        train_lst[len(train_lst) - 1].append(train_lst[len(train_lst) - 2][2])

        lst.append(np.array(train_lst))

    train.append(lst)

    lst = []
    dir_list = os.listdir('Handwriting_Data//'+letter+'//dev')
    for file in dir_list:
        temp = np.loadtxt('Handwriting_Data/'+letter+'/dev/' + file)[1:]
        test_lst = np.array([temp[::2],temp[1::2]]).T
        test_lst = test_lst.tolist()
        for i in range(len(test_lst)-1):# The same extra feature of angle is added to the dev data too
            if test_lst[i+1][0] != test_lst[i][0]: 
                test_lst[i].append(np.arctan((test_lst[i+1][1] - test_lst[i][1])/(test_lst[i+1][0] - test_lst[i][0])))
            else: #If x coordinates of consecutive points are same, then we cannot divide by x2-x1 and hence directly take arctan on +-np.inf
                if test_lst[i+1][1] - test_lst[i][1] > 0: #arctan(np.inf) if y2 - y1 > 0
                    test_lst[i].append(np.arctan(np.inf))
                else:
                    test_lst[i].append(np.arctan(-np.inf)) #arctan(-np.inf) if y1 - y2 > 0
        test_lst[len(test_lst) - 1].append(test_lst[len(test_lst) - 2][2])

        lst.append(np.array(test_lst))

    dev.append(lst)

count = 0
num_frames = 0
for cls in range(len(train)):
    for i in range(len(train[cls])):
        num_frames += len(train[cls][i])
        count += 1
avg_num_frames = math.floor(num_frames/count)

train_all = []
dev_all = []
train_labels = []
dev_labels = []
for cls in range(len(train)):
    for i in range(len(train[cls])):
        train_all.append(signal.resample(train[cls][i],avg_num_frames))
        train_labels.append(cls)

for cls in range(len(dev)):
    for i in range(len(dev[cls])):
        dev_all.append(signal.resample(dev[cls][i],avg_num_frames))
        dev_labels.append(cls)

train_all = np.array(train_all)
dev_all = np.array(dev_all)

mean_train = np.mean(train_all,axis=0)
maxs = np.max(train_all,axis=0)
mins = np.min(train_all,axis=0)
denoms = maxs - mins
train_all = (train_all-mean_train)/denoms
dev_all = (dev_all-mean_train)/denoms


In [8]:
train_extended = []
dev_extended = []
for i in range(len(train_all)):
    lst = []
    for j in range(len(train_all[i])):
        lst.extend(train_all[i][j])
    train_extended.append(np.array(lst))

for i in range(len(dev_all)):
    lst = []
    for j in range(len(dev_all[i])):
        lst.extend(dev_all[i][j])
    dev_extended.append(np.array(lst))

In [9]:
clf = MLPClassifier(solver='adam', activation="tanh",alpha=2,hidden_layer_sizes=(100,50), random_state=1, max_iter = 5000)
clf.fit(train_extended, train_labels)

MLPClassifier(activation='tanh', alpha=2, hidden_layer_sizes=(100, 50),
              max_iter=5000, random_state=1)

In [10]:
predictions = clf.predict(dev_extended)
count_correct = 0
for i in range(len(predictions)):
    if predictions[i] == dev_labels[i]:
        count_correct += 1
print(count_correct/len(dev_labels))

0.98
